### Simulating spectra

Using the XSPEC fake it function to create simulated spectra from the given input. 
Based on this page: https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/python/html/extended.html#fakeit

In [1]:
# importing general packages

import numpy as np
import os, sys
import matplotlib.pyplot as plt

import xspec as xs
xs.Xset.allowPrompting = False #keeps pyxspec from hanging, waiting a response to a prompt
xs.Xset.allowNewAttributes = True

In [2]:
# The units for the model need to be given per bin. Number of bins determined by the 
# input LLE background spectrum.

from astropy.io import fits

hdu = fits.open("bn121225417_LAT-LLE_bkgspectra.bak")
energy = hdu[2].data # this holds energies corresponding to bin ends, in keV
E_MeV_diff = ((energy['E_MAX'] - energy['E_MIN']))/1000. # bin sizes, in MeV
nBins = len(E_MeV_diff)
# print E_MeV_diff

In [3]:
SED10 = np.load('bn121225417_SED10.npy') 

# This SED is produced with ALP_spectrum_norm.py. This is going to be the input model 
# with one free parameter, norm1 (XSPEC assigns normalization as a default fit parameter,
# adding norm1 as it is required to have a manually defined fit parameter. For the future
# analysis, one of the normalizations is always fixed  to 1.)

def ALP(engs, params, flux):
    norm1 = params[0]
    flux[:] = norm1 * SED10[:]/nBins # the model to be given per bin
    # print(len(flux), len(engs)) # these two have to be the same
        
ALPInfo = ("norm1 \"\" 1.1  1e-12  1e-10 1e10 1e12 0.0001",)
xs.AllModels.addPyMod(ALP, ALPInfo, 'add', spectrumDependent=True)

In [4]:
# normalizations, coarse grid
N = np.logspace(np.log10(8.4e-8), np.log10(8.4e2), 30)

xs.AllData.clear() 

# Dummy file (placeholder), format is pha type-II.

s1 = xs.Spectrum('bn121225417_LAT-LLE_srcspectra.pha{1}')
s1 = xs.AllData(1)
s1.response = 'bn121225417_LAT-LLE_weightedrsp.rsp'
s1.background = 'bn121225417_LAT-LLE_bkgspectra.bak{1}'

# spectrum we want to simulate, the spectral type is pha type-I.

for i in range(30):
    # spectral files produced with ALP_spectrum_norm.py and flx2xsp.sh
    s2 = xs.Spectrum('bn121225417_ALP_xspec_%s.pha' %i) 
    s2 = xs.AllData(2)
    s2.response = 'bn121225417_LAT-LLE_weightedrsp.rsp'
    s2.background = 'bn121225417_LAT-LLE_bkgspectra.bak{1}'
    
    # Providing the input model settings for fakeit, but not fitting 
    xs.AllModels.clear()
    m = xs.Model('ALP')
    m.ALP.norm.values = 1.
    m.ALP.norm.frozen = True
    #List of value floats [val,delta,min,bot,top,max].
    m.ALP.norm1.values=[N[i], N[i]*0.01, N[i]*0.1, N[i]*0.95, N[i]*1.05, N[i]*1.9]
    m.ALP.norm1.frozen = True
    
    # Passing the settings
    fs1 = xs.FakeitSettings()
    fs2 = xs.FakeitSettings()
    fs3 = xs.FakeitSettings()
    
    # Dummy files
    fs1.fileName = 'dummy1_0%s.fak' %i
    fs2.fileName = 'dummy2_0%s.fak' %i
    
    # Actual files
    fs3.response = 'bn121225417_LAT-LLE_weightedrsp.rsp'
    fs3.background = 'bn121225417_LAT-LLE_bkgspectra.bak{1}'
    fs3.fileName = 'bn121225417_ALP%s_bkg.fak' %i
    
    # python list to hold 2000 copies of the fs3 settings
    # to go with the 2000 spectra it is making
    
    FS = [fs3]*2000

    # inserting the settings for the loaded spectra at the beginning of the fakeit settings list 
    FS.insert(0, fs2)
    FS.insert(0, fs1)
    
    print(FS[0].fileName, FS[1].fileName, FS[2].fileName)
    xs.AllData.fakeit(len(FS), FS, applyStats=True)

('dummy1_00.fak', 'dummy2_00.fak', 'bn121225417_ALP0_bkg.fak')
('dummy1_01.fak', 'dummy2_01.fak', 'bn121225417_ALP1_bkg.fak')
('dummy1_02.fak', 'dummy2_02.fak', 'bn121225417_ALP2_bkg.fak')
('dummy1_03.fak', 'dummy2_03.fak', 'bn121225417_ALP3_bkg.fak')
('dummy1_04.fak', 'dummy2_04.fak', 'bn121225417_ALP4_bkg.fak')
('dummy1_05.fak', 'dummy2_05.fak', 'bn121225417_ALP5_bkg.fak')
('dummy1_06.fak', 'dummy2_06.fak', 'bn121225417_ALP6_bkg.fak')
('dummy1_07.fak', 'dummy2_07.fak', 'bn121225417_ALP7_bkg.fak')
('dummy1_08.fak', 'dummy2_08.fak', 'bn121225417_ALP8_bkg.fak')
('dummy1_09.fak', 'dummy2_09.fak', 'bn121225417_ALP9_bkg.fak')
('dummy1_010.fak', 'dummy2_010.fak', 'bn121225417_ALP10_bkg.fak')
('dummy1_011.fak', 'dummy2_011.fak', 'bn121225417_ALP11_bkg.fak')
('dummy1_012.fak', 'dummy2_012.fak', 'bn121225417_ALP12_bkg.fak')
('dummy1_013.fak', 'dummy2_013.fak', 'bn121225417_ALP13_bkg.fak')
('dummy1_014.fak', 'dummy2_014.fak', 'bn121225417_ALP14_bkg.fak')
('dummy1_015.fak', 'dummy2_015.fak', 'bn